# COGS 108 - Data Checkpoint

# Names

- Joonsung Park
- James Chung
- Richard Gross
- Madison Hambly
- Colin Lintereur

<a id='research_question'></a>
# Research Question

How do the most popular topics from previous COGS 108 final projects vary per quarter from 2019 to 2021? Also what other trends can we find in past submissions, like changes in word count, and number of graphs used?

# Dataset(s)

- Dataset Name: Past COGS108 final projects
- Link to the dataset: [github.com/COGS108](github.com/COGS108)
- Number of observations: ~200 so far

We looked through the COGS108 github repositories and downloaded the zip file for each quarter of submissions we wanted to include in our analyis. From there, we process each jupyter notebook in the zip file and collect relevant data, parsing it to find the research question of the group. We then generate a csv file for each quarter, and once we have all the csv files of each quarter, we combine them into one final dataset we want to study. There are more quarters and submissions that are not yet included but we may include more if necessary.

# Setup

In [1]:
# importing libraries
import numpy as np
import json
import zipfile
import pandas as pd
import re
import paralleldots
import seaborn as sns
import glob

# copy/paste API key for ParallelDots
API_KEY = 'eTEGyoSgZ8gx1fNiLHrPC4py02eFt4TtWeuQIU4GgB0'

(To avoid reaching the daily usage limit on the API, we each used our own API keys)

We now search through the zip file of a quarter we want to study. We can get the quarter/group/year information from the file names. However, the research question is difficult to find since the formatting of every submission is not always the same. This may cause the file parser to not find any research question or find the wrong cell. However, this is very rare and does not actually ruin our analysis since all we need is a block of text that describes the topic of the project and any block of text near the research question tends to contain relevant information to the topic of the project. For now, even if the found text is not the research question, we will still process it (later we may exclude these projects, or handle them correctly)

We manually ran this code for each project repo/zip in order to use different API keys. 

In [10]:
# setting API key for classifier
paralleldots.set_api_key(API_KEY)

# initializing lists
research_questions = {}
qtr = []
year = []
group = []
# special projects are marked with '_S' at the end of its name
special = []
word_counts = []
index = 0

# reading zip file for a certain quarter
with zipfile.ZipFile("zips/FinalProjects-Sp19-master.zip", "r") as f:
    for i, name in enumerate(f.namelist()):
        # skip files that are not jupyter notebooks
        if '.ipynb' not in name:
            continue
        rf = json.loads(f.read(name))
        word_count = 0
        rq_found = False
        
        # loop through every cell
        for j, cell in enumerate(rf['cells']):
            if cell['cell_type'] == 'markdown':
                for c in cell['source']:
                    # if cell contains the research question header, update lists and extract the research question from the next cell
                    if (rq_found == False) and (re.search(r"(^#.*research question)", c.lower()) != None):
                        #print("RQ FOUND")
                        
                        # extract quarter info from repo name
                        date = re.search(r"-(.*)-", name).group(1)
                        qtr.append(date[:2].upper())
                        year.append(int(date[2:]))

                        # extract group number
                        fname = re.search(r"/.*", name).group(0)
                        group.append(int(re.search(r"[$0-9^]{2,3}", fname)[0]))

                        special.append(False if re.search(r"_S\.ipynb", name) == None else True)
                        # get the research question from the cell AFTER the cell that contains the header 'Research Question'
                        rq = ' '.join(rf['cells'][j+1]['source'])
                        research_questions[str(group[index])+"_"+qtr[index]+"_"+str(year[index])] = rq          
                        rq_found = True
                        index += 1
                    word_count += len(c.replace('#', '').lstrip().split(' '))
        if rq_found:
            word_counts.append(word_count)

After collecting all the data we need from the submissions, we can use a text classifier to find the general topic of the submission. We are currently using an API made by ParallelDots to do this classification for us. See [here](https://apis.paralleldots.com/text_docs/index.html#taxonomy) for details on the API we are using.

In [3]:
# initialize topics list
topic = []

# loop through research questions and find the most confident topic class from topic classifier
for i, (g, q, y) in enumerate(zip(group, qtr, year)):
    response = paralleldots.taxonomy(research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])])
    topic.append(response['taxonomy'][0]['tag'])
    if i == 0:
        print("EXAMPLE:\n", research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])], " :: ", topic[i])

EXAMPLE:
 ### Background & Prior Work
 #### Background
 * Scam callers can spoof their caller ID to make it look like they are from your area.
 * Scam callers can target victims based on certain demographics such as race.
 * Scam callers sometimes do not take people off their call lists when requested.
 
 #### Prior Work
 We found two projects similar to ours: **The Large-Scale Analysis of Technical Support Scams** and **The Correlates of Susceptibility to Scams in Older Adults Without Dementia**.
 
 **The Large-Scale Analysis of Technical Support Scams** study learned that scammers abuse specific browser APIs to make it hard for users to navigate away from a technical support scam page, identified the social engineering techniques used, the remote administration tools abused, and the amount of money that scammers are charging. Technical support scammers were also based in call centers in English-speaking countries with low wages.
 
 **The Correlates of Susceptibility to Scams in Older

Now that we have stored the topics of the submissions, we can run a sanity check to make sure that all of our list are the same length and then export the data to a csv file.

In [8]:
# double check that all the lists are the same length
print(len(group), len(qtr), len(year), len(topic), len(special), len(word_counts))

# generate data frame and output it to a csv file
d = {'Group':group, 'Quarter':qtr, 'Year':year, 'Topic':topic, 'Special':special, 'Word_Count':word_counts}
df = pd.DataFrame(data = d)
output = df.to_csv(path_or_buf="./csvs/"+qtr[0] + "20" + str(year[0])+".csv")

110 110 110 110 110 110


# Data Cleaning

Since we are collecting the data ourselves, there is not much work to be done cleaning the data after collecting all of the csv files for each quarter and merging them together into one large dataset. One column we do need to remove however is a column named 'Unnamed: 0'. This column is an artifact of reading csv files that store index information for each dataset. This column is redundant in our new dataset so we can remove it.

In [9]:
# using glob to get all csvs
files = glob.glob('./csvs/*.csv')
df_total = pd.DataFrame()
entries = 0

# read all csvs to dataframes and combine into one df
for f_name in files:
    csv = pd.read_csv(f_name)
    df_total = df_total.append(csv)
    entries = entries + len(csv.index)

# remove artifact 'Unnamed'
df_total = df_total.drop('Unnamed: 0', axis=1)

assert(len(df_total.index) == entries)
df_total.head()

,Group,Quarter,Year,Topic,Special,Word_Count
0,1,FA,2020,ARTS & CULTURE,False,7002
1,2,FA,2020,BUSINESS,False,8689
2,4,FA,2020,TECH,False,1864
3,5,FA,2020,EDUCATION,False,5483
4,6,FA,2020,GREEN,False,4595
